In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import torchmetrics
import optuna

/usr/local/lib/python3.12/dist-packages/sqlalchemy/orm/query.py:195: SyntaxWarning: "is not" with 'tuple' literal. Did you mean "!="?
  if entities is not ():


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5), #melhorar os dados de treino
    transforms.RandomRotation(10),
    transforms.ColorJitter(0.2, 0.2, 0.2),
    
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.5, 0.5, 0.5),
        std=(0.5, 0.5, 0.5)
    )  
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.5, 0.5, 0.5),
        std=(0.5, 0.5, 0.5)
    )
])

train_dataset = datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=train_transform
)

test_dataset =datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=val_transform
)

train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=4, #núcleos cpu
    pin_memory=True
)

test_loader =DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=4, #núcleos cpu
    pin_memory=True
)

100%|██████████| 170M/170M [00:02<00:00, 80.5MB/s]


In [4]:
class FlexibleCNN(nn.Module):
    def __init__(self,
                 num_blocks,
                 filters_per_block,
                 kernel_sizes,
                 dropouts,
                 num_fc_units,
                 fc_units):
        super().__init__()
        
        # --- PARTE 1: Extração de Características (Convoluções) ---
        convolution_layers = []
        for i in range(num_blocks):
            out_channels = filters_per_block[i]
            
            convolution_layers.append(
                nn.LazyConv2d(
                    out_channels=out_channels,
                    kernel_size=kernel_sizes[i],
                    padding=kernel_sizes[i] // 2
                )
            )
            convolution_layers.append(nn.BatchNorm2d(filters_per_block[i]))
            convolution_layers.append(nn.ReLU())
            convolution_layers.append(nn.MaxPool2d(2))
            

        self.feature_extractor = nn.Sequential(*convolution_layers)

        # --- PARTE 2: Classificador (Camadas Densas) ---
        fc_layers = []
        for i in range(num_fc_units):
            fc_layers.append(nn.LazyLinear(fc_units[i]))
            fc_layers.append(nn.ReLU())
            fc_layers.append(nn.Dropout(dropouts[i]))
            
        self.classifier = nn.Sequential(*fc_layers)
        # Camada de Saída Final (Fixa em 10)
        self.output = nn.LazyLinear(10)

    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        x = self.output(x)
        return x
        

In [5]:
def objective(trial):
    num_blocks = trial.suggest_int("num_blocks", 1, 3)

    filters_per_block = [
        trial.suggest_int(f"filters_block_{i}", 32, 128)
        for i in range(num_blocks)
    ]

    kernel_sizes = [
        trial.suggest_categorical(f"kernel_size_{i}", [3, 5])
        for i in range(num_blocks)
    ]

    num_fc_layers = trial.suggest_int("num_fc_layers", 1, 3)

    fc_units = [
        trial.suggest_int(f"fc_units_{i}", 64, 256)
        for i in range(num_fc_layers)
    ]

    dropouts = [
        trial.suggest_float(f"dropout_{i}", 0.1, 0.4)
        for i in range(num_fc_layers)
    ]

    model = FlexibleCNN(num_blocks, filters_per_block, kernel_sizes, dropouts, num_fc_layers, fc_units).to(device)

    # Dummy pass para inicializar Lazy layers
    with torch.no_grad():
        model(torch.randn(1, 3, 32, 32).to(device))

    # Métrica
    acc_metric = torchmetrics.Accuracy(task="multiclass", num_classes=10).to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0005)
    loss_function = nn.CrossEntropyLoss()
    
    # Loop de Treino
    for epoch in range(10):
        model.train()
    
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)
    
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()
    
    # Validação Final
    model.eval()
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            acc_metric.update(outputs, labels)
        
    return acc_metric.compute().item()

In [6]:
# Criamos o estudo
study = optuna.create_study(direction="maximize")

# Iniciamos as tentativas (ex: 10 vezes)
study.optimize(objective, n_trials=15)

print("\n--- RESULTADO DA OTIMIZAÇÃO ---")
print(f"Melhor Acurácia: {study.best_value:.4f}")
print("Melhores Parâmetros:", study.best_params)

[I 2025-12-28 13:58:53,786] A new study created in memory with name: no-name-d9d1fc7b-ffc1-4611-a570-d6638cb4306b
[I 2025-12-28 14:01:18,568] Trial 0 finished with value: 0.6586999893188477 and parameters: {'num_blocks': 1, 'filters_block_0': 100, 'kernel_size_0': 3, 'num_fc_layers': 3, 'fc_units_0': 152, 'fc_units_1': 135, 'fc_units_2': 128, 'dropout_0': 0.10116223895069545, 'dropout_1': 0.13536348001670429, 'dropout_2': 0.1030262788710622}. Best is trial 0 with value: 0.6586999893188477.
[I 2025-12-28 14:03:44,988] Trial 1 finished with value: 0.7587000131607056 and parameters: {'num_blocks': 3, 'filters_block_0': 92, 'filters_block_1': 64, 'filters_block_2': 36, 'kernel_size_0': 5, 'kernel_size_1': 5, 'kernel_size_2': 3, 'num_fc_layers': 2, 'fc_units_0': 108, 'fc_units_1': 164, 'dropout_0': 0.27406376007190386, 'dropout_1': 0.3942802220660345}. Best is trial 1 with value: 0.7587000131607056.
[I 2025-12-28 14:06:14,349] Trial 2 finished with value: 0.7549999952316284 and parameters: 


--- RESULTADO DA OTIMIZAÇÃO ---
Melhor Acurácia: 0.7889
Melhores Parâmetros: {'num_blocks': 3, 'filters_block_0': 51, 'filters_block_1': 79, 'filters_block_2': 107, 'kernel_size_0': 3, 'kernel_size_1': 5, 'kernel_size_2': 3, 'num_fc_layers': 2, 'fc_units_0': 177, 'fc_units_1': 210, 'dropout_0': 0.39605198371528916, 'dropout_1': 0.3090776647539773}


In [7]:
# 1. Extrair os valores do melhor trial
bp = study.best_params

# 2. Reconstruir as listas que o modelo espera
best_num_blocks = bp["num_blocks"]

best_filters = [
    bp[f"filters_block_{i}"]
    for i in range(best_num_blocks)
]

best_kernels = [3 for _ in range(best_num_blocks)]  # se kernel foi fixado

best_num_fc = bp["num_fc_layers"]

best_fc_units = [
    bp[f"fc_units_{i}"]
    for i in range(best_num_fc)
]

best_dropouts = [
    bp[f"dropout_{i}"]
    for i in range(best_num_fc)
]

# 3. Instanciar o modelo vencedor
best_model = FlexibleCNN(
    num_blocks=best_num_blocks,
    filters_per_block=best_filters,
    kernel_sizes=best_kernels,
    dropouts=best_dropouts,
    num_fc_units=best_num_fc,
    fc_units=best_fc_units
)

# 4. Inicializar Lazy layers
with torch.no_grad():
    best_model(torch.randn(1, 3, 32, 32))

best_model.to(device)
print(f"Modelo movido com sucesso para: {device}")

Modelo movido com sucesso para: cuda


In [8]:
acc_metric = torchmetrics.Accuracy(task="multiclass", num_classes=10).to(device)
    
optimizer = optim.Adam(best_model.parameters(), lr=0.001, weight_decay=0.0005)
loss_function = nn.CrossEntropyLoss()

for epoch in range(50):
    best_model.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
    
        optimizer.zero_grad()
        outputs = best_model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    print("training epoch: ", epoch)
    print("loss:", running_loss/len(train_loader))

training epoch:  0
loss: 1.519767015867526
training epoch:  1
loss: 1.1829857089940239
training epoch:  2
loss: 1.0430213325773663
training epoch:  3
loss: 0.9612734090641636
training epoch:  4
loss: 0.9085509685604164
training epoch:  5
loss: 0.8603341475777004
training epoch:  6
loss: 0.8210163944974884
training epoch:  7
loss: 0.791637986288656
training epoch:  8
loss: 0.7618383250349318
training epoch:  9
loss: 0.7361533551874673
training epoch:  10
loss: 0.7225006070664471
training epoch:  11
loss: 0.7030963406843298
training epoch:  12
loss: 0.6845229756268089
training epoch:  13
loss: 0.6706946201793983
training epoch:  14
loss: 0.6554107630572965
training epoch:  15
loss: 0.6491849651879362
training epoch:  16
loss: 0.6401425405688907
training epoch:  17
loss: 0.6353043811324307
training epoch:  18
loss: 0.6223413989214641
training epoch:  19
loss: 0.6158870584748285
training epoch:  20
loss: 0.6038862667272767
training epoch:  21
loss: 0.6010135178599516
training epoch:  22
lo

In [9]:
best_model.eval()
with torch.no_grad():
    for image, label in test_loader:
        images = image.to(device)
        labels = label.to(device)
        output = best_model(images)
        acc_metric.update(output, labels)

print(f"Acurácia: {acc_metric.compute() * 100 :.4f}")

Acurácia: 81.3300
